# Proyecto A: Optimización en la Planeación de Transporte Vehicular Urbana Para LogistiCo

## Integrantes

- Barrera Toro, Javier Steven
- Borbon Holguin, Luis Alfredo
- Rolon Toloza, Julian Santiago

## 1.  Formulación Matemática del Modelo

### 1.1. Definición de Conjuntos

- C: Conjunto de centros de distribución:

$$C = \{C1, C2, C3\}$$
- Z: Conjunto de zonas de entrega.

$$Z = \{Z1, Z2, Z3\}$$
- N Conjunto de todos los nodos.

$$N = C \cup Z$$
- T: Conjunto de vehículos disponibles.

$$ T = \{T1, T2, T3\} $$
- R: Conjunto de rutas posibles entre nodos, definidos por:

$$ R = \{(i, j) ∈ N × N | i ̸= j\} $$ 

### 1.2. Definición de parametros:

TODO: Agregar justificación.

https://www.motor.com.co/actualidad/Nueva-alza-en-la-gasolina-en-Colombia-aumento-95-pesos-en-febrero-2025-20250203-0002.html

https://cabify.com/co/tarifas/bogota#p-envios

Los costos que se asumen constantes durante todo el proceso son los siguientes. 

- Costo de mantenimiento ($M$): \$600 por km.
- Precio del combustible ($G$): \$4.145 por km.
- Tarifa del servicio ($P$): \$600 por km
- Consumo del combustible: 0,1 litros por km. Lo que significa 0,1 * G = 0,1*$15.000 = 1,500$
- Costo total por km = M+G+1.500

### 1.3. Variables de decision:

1. Distancia $d_{ij}$: Es la distancia en kilometros de los nodos i y j.

2. Demanda $s_j$ representa la demanda en unidades del cliente j.

sj= 50 si j=Z1
sj= 65 si j=Z3
sj= 80 si j=Z2
sj=0 en los centros de distribucion (Z)

3. Stock: Disponibilidad del inventario en el centro de distribucion C.

DC1 = 20000
DC2 = 50000
DC3 = 30000

5. Capacidad en el trasporte: Corresponde a la capacidad maxima que tienen los vehiculos para transportar unidades.

CV1=100
CV2=80
CV3=150


1. Ruta ($X_{tij}$) : Variable binaria, si vale 1 si el vehículo t ∈ T viaja directamente del nodo i al nodo j, y 0 en caso contrario.
2. Cantidad de inventario: Cantidad de unidades transportado por el vehículo t en el arco (i, j).
3. Deposito: Variables binarias, en las cuales 1 representa  si el vehículo t inicia en el deposito, y 0 de lo contrario.


### 1.4. Restricciones

**TODO** Cambiar 3 para que se parezca a la 5. Eliminación de subtours del problema de TSP. 8 revisarla en detalle, pero dijo que no valia.

1. Visita de los clientes: Se debe garantizar que se visitan a los clientes al menos una vez en recorrido.
$$
\sum_{t \in T} \sum_{\substack{i \in N }} x_{tij} = 1
$$

2. Flujos en nodos: Se debe garantizar la demanda de las rutas que pasen por los clientes
$$
\sum_{\substack{i \in N}} f_{tij}
- \sum_{\substack{k \in N }} f_{tjk}
= s_j \sum_{\substack{i \in N}} x_{tij}
$$


3. Ruta: La capacidad del inventario en el arco (i, j) no puede superar la capacidad del vehiculo que tiene el recorrido:
$$
f_{tij} \leq \text{capacidad}_t \, x_{tij}
$$

4. Demanda operativa: Cantidad de unidades desde el centro de distribucón no puede superar el stock disponible:

$$
\sum_{t \in } \sum_{\substack{j \in N}} f_{tdj} \leq s_d
$$

5. Rango: La distancia total recorrida por cada transporte no puede exceder su rango operativo:

$$
\sum d_{ij} \, x_{tij} \leq r_v
$$

6. Asignacion de inicio y fin: Cada transporte debe iniciar y terminar su recorrido:
$$
\sum_{} \text{inicio}_{td} \leq 1 ,
\qquad \sum_{} \text{fin}_{td} \leq 1
$$
7. Asignacion de dispositivo y ruta: Se debe garantizar que la elección del deposito este vinculada a la ruta:
$$
\sum_{\substack{j \in N}} x_{tdj} = \text{inicio}_{td} \\
\sum_{\substack{i \in N}} x_{vid} = \text{fin}_{td}
$$
8. Asignacion de vehiculo y demanda: La demanda total atendida no puede superar su capacidad, y solo si efectivamente inicia su ruta desde algún depósito.
$$
\sum_{} s \left( \sum_{\substack{i \in N}} x_{ti} \right)
\leq \text{capacidad}_t \left( \sum_{} \text{inicio}_{td} \right)
$$

## 2. Función Objetivo y Proceso de indagación

Para definir esta función objetivo, se llevó a cabo un análisis relacionado con la optimización de rutas y problemas logísticos. La formulación se diseñó con el objetivo de reducir al máximo los costos de transporte, los cuales están determinados por la distancia que se recorre y los costos operativos como tarifas de flete, mantenimiento y consumo de combustible.
El propósito es reducir al mínimo el costo total de transporte. Este costo se calcula teniendo en cuenta el valor por kilómetro recorrido, multiplicado por la distancia total que recorre cada vehículo en los trayectos seleccionados.

La función objetivo está definida como:

$$
\min\left(\sum_{t \in T} \sum_{(i,j) \in R} c_{ij} \cdot d_{ij} \cdot x_{tij} \right)
$$

## 3. Preprocesamiento de datos y análisis

**TODO** Convertir los datos de LAT, LONG a tiempo.

## 4. Ejemplos ilustrativos

**TODO**: Que se cumplan todas y otro ejemplo en el que una restricción no se cumpla.

## 5. Justificación y discusión

**TODO**: Suposiciones que hay que hacer al modelo para simplificarlo. 